# Bertrand's paradox

In [ ]:
import os
import sys

import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['text.usetex'] = True

from typing import Callable, List

In [ ]:
def circle(num : int=100) -> np.ndarray:
    """Creates an array from the Cartesian coordinates of points on the
    circumference of a circle with an arbitrary sampling frequency.

    Parameters
    ----------
    num : int
      Sampling frequency of the circle's circumference

    Returns
    -------
    cc : numpy.ndarray
      Points on the circumference of a circle with R=1.
    """
    phi = np.linspace(0, 2*np.pi, num=num)
    # Get Cartesian coprdinates of the circle's points
    cc = np.stack((np.sin(phi), np.cos(phi)), axis=1)
    return cc

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

# Plot the circle
cc = circle()
ax.plot(cc[:, 0], cc[:, 1], color='k', linewidth=2)

plt.show()

In [ ]:
def bertrand_1(N : int=1) -> np.ndarray:
    """Implements the "random endpoints" method in Bertrand's paradox.

    Tha algorithm chooses two random points on the circumference of the
    circle. These points will serve as the endpoints for the random
    chord.

    Parameters
    ----------
    N : int
        Number of chords to be randomly generated.

    Returns
    -------
    cc : numpy.ndarray
      The Cartesian coordinates of the two endpoints of the chord.
    """
    # Select two random endpoints for a chord
    phi = 2 * np.pi * np.random.random(size=N*2).reshape(-1, 2)
    # Get Cartesian coordinates of endpoints
    cc = np.stack((np.sin(phi), np.cos(phi)), axis=2)
    return cc

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

# Plot the circle
cc = circle()
ax.plot(*cc.T, color='k', linewidth=2)

# Plot the random chords
cc = bertrand_1(N=200)
ax.plot(*cc.T, color='r', linewidth=1, alpha=0.5)

plt.show()

In [ ]:
def bertrand_2(N : int=1) -> np.ndarray:
    """Implements the "random radial point" method in Bertrand's paradox.

    The algorithm randomly selects a radius of the circle, then chooses
    a random point on this radius. This point will serve as the midpoint
    for the random chord.

    Parameters
    ----------
    N : int
        Number of chords to be randomly generated.

    Returns
    -------
    cc : numpy.ndarray
      The coordinates of the two endpoints of the chord.
    """
    # Select a random radius of the circle
    phi = 2 * np.pi * np.random.random(size=N)
    # Get a random point on this radius that will serve as the midpoint
    mid = np.random.random(size=N)
    # Get endpoints of the random chord from the random midpoint
    t = np.stack((phi + np.arccos(mid), phi - np.arccos(mid)), axis=1)
    # Get Cartesian coordinates of endpoints
    cc = np.stack((np.sin(t), np.cos(t)), axis=2)
    return cc

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

# Plot the circle
cc = circle()
ax.plot(*cc.T, color='k', linewidth=2)

# Plot the random chords
cc = bertrand_2(N=200)
ax.plot(*cc.T, color='r', linewidth=1, alpha=0.5)

plt.show()

In [ ]:
def bertrand_3(N : int=1) -> np.ndarray:
    """Implements the "random midpoint" method in Bertrand's paradox.

    The algorithm randomly selects a point inside the circle. This point
    will serve as the midpoint for the random chord.

    Returns
    -------
    cc : numpy.ndarray
      The coordinates of the two endpoints of the chord.
    """
    # Select a random radius of the circle
    phi = 2 * np.pi * np.random.random(size=N)
    # Select a random (mid)point inside the circle (normed with sqrt to
    # get uniformly distributed points inside the circle)
    mid = np.sqrt(np.random.random(size=N))
    # Get endpoints of the random chord from the random midpoint
    t = np.stack((phi + np.arccos(mid), phi - np.arccos(mid)), axis=1)
    # Get Cartesian coordinates of endpoints
    cc = np.stack((np.sin(t), np.cos(t)), axis=2)
    return cc

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6), dpi=120, facecolor='black',
                       subplot_kw=dict(facecolor='black'))

# Plot the circle
cc = circle()
ax.plot(*cc.T, color='white', linewidth=2, zorder=2)

# Plot the random chords
cc = bertrand_3(N=200)
ax.plot(*cc.T, color='r', linewidth=1, alpha=0.5, zorder=1)

P = calculate_p(cc)
P_text = f'Method \#{f.__name__.split("_")[1]}\n$P = {P:.3f}$'
ax.text(0.01, 0.99, P_text, color='white', fontweight='bold',
        transform=ax.transAxes, fontsize=12, verticalalignment='top')

plt.show()

In [ ]:
def calculate_p(cc):
    """Calculates what percentage of the chords are longer than the
    side of the inscribed triangle of the circle they belong to.

    Parameters
    ----------
    cc : numpy.ndarray of shape (-1, 2, 2)
      List of coordinates of the endpoints of chords.

    Returns
    -------
    P : float
      Percentage of chords longer than sqrt(3).
    """
    d = np.linalg.norm(cc[:, 1, :] - cc[:, 0, :], axis=1)
    P = np.sum(d > np.sqrt(3)) / len(cc)

    return P

In [ ]:
def plot_midpoints(f : Callable, N : int, **kwargs) -> None:
    """
    Parameters
    ----------
    f : callable
        The function generating the random chords.
    N : int
        Number of chords to be generated.
    """

    fig, ax = plt.subplots(figsize=(5, 5), dpi=120, facecolor='black',
                           subplot_kw=dict(facecolor='black'))
    ax.set_aspect('equal')
    ax.axis('off')

    ax.plot(*circle(num=100).T,
            color='white', alpha=0.9, lw=2, zorder=2)

    cc = f(N)
    ax.scatter(*np.mean(cc, axis=1).T, **kwargs)

    P = calculate_p(cc)
    #P_text = f'Method \#{f.__name__.split("_")[1]}\nP = {P:.3f}'
    #ax.text(0.01, 0.99, P_text, color='white', fontweight='bold',
    #        transform=ax.transAxes, fontsize=12, verticalalignment='top')

    ax.set_xticks([])
    ax.set_yticks([])

    os.makedirs('./out/', exist_ok=True)
    plt.savefig(f'{os.path.join("./out/", f.__name__)}-midpoints.png',
                dpi=120, pad_inches=0.5, bbox_inches='tight')
    plt.close(fig)

    return

In [ ]:
def plot_chords(f : Callable, N : int, **kwargs) -> None:
    """
    Parameters
    ----------
    f : callable
        The function generating the random chords.
    N : int
        Number of chords to be generated.
    """
    fig, ax = plt.subplots(figsize=(5, 5), dpi=120, facecolor='black',
                           subplot_kw=dict(facecolor='black'))
    ax.set_aspect('equal')
    ax.axis('off')

    ax.plot(*circle(num=100).T,
            color='white', alpha=0.9, lw=2, zorder=2)

    cc = f(N)
    ax.plot(*cc.T, **kwargs)

    P = calculate_p(cc)
    #P_text = f'Method \#{f.__name__.split("_")[1]}\nP = {P:.3f}'
    #ax.text(0.01, 0.99, P_text, color='white', fontweight='bold',
    #        transform=ax.transAxes, fontsize=12, verticalalignment='top')

    ax.set_xticks([])
    ax.set_yticks([])

    os.makedirs('./out/', exist_ok=True)
    plt.savefig(f'{os.path.join("./out/", f.__name__)}-chords.png',
                dpi=120, pad_inches=0.5, bbox_inches='tight')
    plt.close(fig)

    return

In [ ]:
N = 500
fs = [bertrand_1, bertrand_2, bertrand_3]
cs = ['#FF00FF', '#00FFFF', '#FFCC00']
als = [0.5, 0.6, 0.6]
for f, c, a in zip(fs, cs, als):
    plot_chords(f, N, color=c, alpha=a, lw=0.5, zorder=1)
    plot_midpoints(f, 10*N, color=c, alpha=0.6, s=1**2, zorder=1)

In [ ]:
def main() -> None:
    n = int(sys.argv[1])

    fs = [bertrand_1, bertrand_2, bertrand_3]
    cs = ['#FF00FF', '#00FFFF', '#FFCC00']
    als = [0.5, 0.6, 0.6]
    for f, c, a in zip(fs, cs, als):
        plot_chords(f, N, color=c, alpha=a, lw=0.5, zorder=1)
        plot_midpoints(f, 10*N, color=c, alpha=0.6, s=1**2, zorder=1)

    return


if __name__ == '__main__':
    main()